# <center> Insertion des données dans la BDD </center>

In [ ]:
import pandas as pd
import mysql.connector as msql
from mysql.connector import Error
from tqdm.notebook import tqdm_notebook

## Chargement du csv

In [7]:
df = pd.read_csv("partie_1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Auteur   100000 non-null  object
 1   Titre    100000 non-null  object
 2   Edition  100000 non-null  object
 3   Date     100000 non-null  int64 
 4   ISBN     100000 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.8+ MB


### Insertion dans les tables secondaires

In [16]:
def insertion_tables(df_name):
    df = pd.read_csv(df_name + '.csv')
    
    # Insérer les auteurs 
    for auteur in tqdm_notebook(df.Auteur):
        
        cursor = conn.cursor()
        # Vérifier si l'auteur existe déjà dans la base
        select_auteur_query = "SELECT id_auteur FROM auteur WHERE nom_auteur = %s"
        cursor.execute(select_auteur_query, (auteur,))
        result = cursor.fetchone()

        # Si l'auteur n'existe pas, on l'insère dans la base
        if not result:
            insert_auteur_query = "INSERT INTO auteur (nom_auteur) VALUES (%s)"
            cursor.execute(insert_auteur_query, (auteur,))
            id_auteur = cursor.lastrowid
        else:
            id_auteur = result[0]

        # Valider les changements dans la base
        conn.commit()
        
    # insérer les maisons d'éditions
    for edition in tqdm_notebook(df.Edition):
        
        cursor = conn.cursor()
        
        select_edition_query = "SELECT id_edition FROM edition WHERE nom_edition = %s"
        cursor.execute(select_edition_query, (edition,))
        result = cursor.fetchone()

        if not result:
            insert_edition_query = "INSERT INTO edition (nom_edition) VALUES (%s)"
            cursor.execute(insert_edition_query, (edition,))
            id_edition = cursor.lastrowid
        else:
            id_edition = result[0]

        # Valider les changements dans la base
        conn.commit()
        
    # Insérer les dates
    for date in tqdm_notebook(df.Date):
        
        cursor = conn.cursor()
        
        select_date_query = "SELECT id_date FROM date WHERE date = %s"
        cursor.execute(select_date_query, (date,))
        result = cursor.fetchone()

        if not result:
            insert_date_query = "INSERT INTO date (date) VALUES (%s)"
            cursor.execute(insert_date_query, (date,))
            id_date = cursor.lastrowid
        else:
            id_date = result[0]

        # Valider les changements dans la base
        conn.commit()

In [17]:
connexion("test_pco")
insertion_tables("partie_1")
deconnexion()

Connexion réussie !


  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

Déconnexion réussie.


In [121]:
connexion("test_pco")
data_tables("partie_1")
deconnexion()

Connexion réussie !


  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

Insertions data : ok !
Déconnexion réussie.


### Insertion dans la table principale

In [18]:
def data(df_name):
    import pandas as pd
    # dataframe :
    df = pd.read_csv(df_name + '.csv')
    # insert data
    try : 
        cursor = conn.cursor()
        
        for row in tqdm_notebook(df.itertuples()):
            
        
                cursor.execute(
                '''INSERT INTO Refs (ISBN, titre, auteur, date, edition)
                    VALUES (%s, %s,
                    (SELECT id_auteur FROM auteur WHERE nom_auteur = %s LIMIT 1),
                    (SELECT id_date FROM date WHERE date = %s LIMIT 1),
                    (SELECT id_edition FROM edition WHERE nom_edition = %s LIMIT 1));''',
                (row.ISBN, row.Titre, row.Auteur, row.Date, row.Edition))
                
          


        conn.commit()
        print("Insertions data : ok !")
            
            
    
    except Error as e:
        print("Error while connecting to MySQL", e)

In [19]:
connexion("test_pco")
data("partie_1")
deconnexion()

Connexion réussie !


0it [00:00, ?it/s]

Insertions data : ok !
Déconnexion réussie.


## Requêtes 

In [26]:
connexion("test_pco")
# Create a cursor object
cursor = conn.cursor()

# Execute the SELECT statement
ISBN = 9780007498130
query = "SELECT * FROM refs WHERE ISBN = %s"
cursor.execute(query, (ISBN,))

# Fetch the result
result = cursor.fetchone()

# Check if the result is None (indicating the book does not exist)
if result:
    print("Record exists:", result)
else:
    print("Record does not exist.")

# Close the cursor and connection
deconnexion()

Connexion réussie !
Record exists: ('9780007498130', 'The Wilful Princess and the Piebald Prince', 17797, 94237, 13)
Déconnexion réussie.


## Requête pour récupérer les informations d'un livre via l'ISBN

In [38]:
connexion("test_pco")

cursor = conn.cursor()

ISBN = 9780007498130
requete = '''SELECT ISBN, titre, date.date, edition.nom_edition, auteur.nom_auteur
FROM refs
JOIN date ON refs.date = date.id_date
JOIN edition ON refs.edition = edition.id_edition
JOIN auteur ON refs.auteur = auteur.id_auteur
WHERE refs.ISBN = %s;'''
cursor.execute(requete, (ISBN,))

# Fetch the result
result = cursor.fetchone()

# Check if the result is None (indicating the book does not exist)
if result:
    print("Record exists:", result)
else:
    print("Record does not exist.")

# Close the cursor and connection
deconnexion()

Connexion réussie !
Record exists: ('9780007498130', 'The Wilful Princess and the Piebald Prince', 2013, 'HarperVoyager', 'Robin Hobb')
Déconnexion réussie.


In [57]:
print(f"{result[4]}, {result[1]}, {result[3]}, {result[2]}, {int(result[0])}.", )

Robin Hobb, The Wilful Princess and the Piebald Prince, HarperVoyager, 2013, 9780007498130.
